'''
Created on Sep 20, 2019

@author: Gias
'''

Topic Model Guide

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

Connect to drive to get and send data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Install packages

In [ ]:
!pip install hickle
!pip install tinydb
!pip install django
!pip install corpus
!pip install pyLDAvis

# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# !unzip mallet-2.0.8.zip

In [ ]:
import sys
# !{sys.executable} -m pip install -U pip setuptools wheel
# !{sys.executable} -m pip install -U spacy
# !{sys.executable} -m spacy download en_core_web_sm
# !{sys.executable} -m spacy download en
# !{sys.executable} -m pip install gensim==3.8.3
# !{sys.executable} -m pip uninstall gensim -y pyserial
print(gensim.__version__)

Import packages

In [ ]:
import os 
import numpy as np
import pandas as pd
import pickle as pickle
from pprint import pprint
from lxml import etree
import hickle as hkl
from tinydb import TinyDB, Query
import marshal
from bs4 import BeautifulSoup
#from nltk import word_tokenize
from nltk.stem.porter import *
#from nltk.corpus import stopwords 
import nltk
# nltk.download('stopwords')
# nltk.download('gutenberg')
# nltk.download('genesis')
# nltk.download('inaugural')
# nltk.download('nps_chat')
# nltk.download('webtext')
# nltk.download('treebank')
from nltk.book import *
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
import csv
from gensim import corpora
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem import WordNetLemmatizer 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

dictSW = []
for w in stopwords:
    dictSW.append(w)
malletStopwords = set()
try:
#     malletStopLists = "/content/mallet-2.0.8/stoplists/en.txt"
    malletStopLists = "mallet-2.0.8/stoplists/en.txt"
    with open(malletStopLists, 'rb') as f:
        lines = f.readlines()
        for line in lines:
            malletStopwords.add(line.strip())
    for w in malletStopwords:
        w = str(w).strip("b'")
        dictSW.append(w)
except:
    pass

import re
import os
import string
from urllib.parse import urlparse, urlunsplit, urlsplit
import unicodedata
import nltk
import django
from django.conf import settings
import shutil
#Gensim
import gensim
import gensim.corpora as gensim_corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

from collections import Counter
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
# update this path where Mallet is downloaded.
#os.environ['MALLET_HOME'] = r'C:\dev\opinion\nlpmodels\mallet-2.0.8'
# os.environ['MALLET_HOME'] = "/content/mallet-2.0.8" #CHANGE

#mallet_path = r'C:\\dev\opinion\nlpmodels\mallet-2.0.8\bin\mallet' # update this path
# mallet_path = "/content/mallet-2.0.8/bin/mallet"


------------------------------------------------------------

Add words to blacklist

In [ ]:
dictSW.append('vulnerability')
dictSW.append('vulnerable')
dictSW.append('due')
dictSW.append('include')
dictSW.append('aka')
dictSW.append('remote')
dictSW.append('attacker')
dictSW.append('attackers')
dictSW.append('attack')
dictSW.append('quot')
dictSW.append('would')
dictSW.append('could')
dictSW.append('able')
dictSW.append('vulnerabilities')
dictSW.append('Microsoft')
dictSW.append('Oracle')
dictSW.append('oracle')
dictSW.append('Windows')
dictSW.append('Android')
dictSW.append('wordpress')
dictSW.append('Version')
dictSW.append('version')
dictSW.append('Apple')
dictSW.append('apple')
dictSW.pop(0)

print(len(dictSW))

dictSW

Get all descriptions from NVD with 0.95 similarity or less.
Get CWEs, CVEs and the number of NVDs per year.
Words are cleaned, stemmed, and lemmatized to obtain a corpus and id2word

In [ ]:
def clean_string(text):
  text = ''.join([word for word in text if word not in string.punctuation])
  text = text.lower()
  text = ' '.join([word for word in text.split() if word not in dictSW])

  return text

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#db= "/content/drive/MyDrive/DISA Labs Research 2021/Vulnerability Databases/New_NVD/0.8/Cosine_NVD_2011-2021_0.8.csv"
# db = "/content/drive/MyDrive/Vulnerability Databases/New_NVD/0.95/Cosine_NVD_2011-2021_0.95_reject.csv"
db = "NVD_2011-2021.csv"

fields = []
rows = []

cve = []
cwe = []
data = []

years = {
    "2011": 0,
    "2012": 0,
    "2013": 0,
    "2014": 0,
    "2015": 0,
    "2016": 0,
    "2017": 0,
    "2018": 0,
    "2019": 0,
    "2020": 0,
    "2021": 0,
}

# Get rows from designated CSV file

with open(db, 'r', encoding='utf-8-sig') as csvfile:
    csvreader = csv.reader(csvfile)
    fields = next(csvreader)
    for row in csvreader: 
        rows.append(row)
    
    for j in rows:
        if j[2].find("** REJECT **") == -1:
            cve.append(j[0])
            data.append(j[2])
            cwe.append(j[4])
            years.update({j[1].split("/")[2]: (years[j[1].split("/")[2]] + 1)})

print("Complete")
print(len(data))

cleaned = list(map(clean_string, data))
# print(cleaned[0])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(cleaned))

print("data_words: ", len(data_words))
print(data_words[0])

# stem_words = []

# for i in data_words:
#     stem_words.append(list(map(ps.stem, i))) #ps is porter stemmer

# print("stemming")
# print(stem_words[0])

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# see a trigram example
# print(trigram_mod[bigram_mod[data_words[7]]])

# (1) without stemming 
# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# (2) with stemming
# # Form Bigrams
# data_words_bigrams = make_bigrams(stem_words)

# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# # python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# # Create Dictionary
# id2word = corpora.Dictionary(data_lemmatized)

# # Create Corpus
# texts = data_lemmatized

# # Term Document Frequency
# corpus = [id2word.doc2bow(text) for text in texts]

# # View
# print(corpus[:1])

Get LDA mallet

In [ ]:
# os.environ.update({'MALLET_HOME':r'PATH'}) #CHANGE
# mallet_path = r'PATH' #CHANGE
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=18, id2word=id2word)
# Show Topics
pprint(ldamallet.show_topics(18, formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Get the dominant topic of each description

In [ ]:
def format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data, cveids=cve, cwes=cwe):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    
    # Add original cveids to the end of the output
    contents2 = pd.Series(cveids)
    sent_topics_df = pd.concat([sent_topics_df, contents2], axis=1)

    # Add original cwes to the end of the output
    contents3 = pd.Series(cwes)
    sent_topics_df = pd.concat([sent_topics_df, contents3], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data, cveids=cve, cwes=cwe)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'CVE-id', 'CWE']

# Show
df_dominant_topic.head(10)
print(df_dominant_topic.to_csv("labelling.csv"))

SECTION 1: Label topics

In [ ]:
all_topics = []

Topics = {
    "Topic ID": None,
    "Topic Label": None,
    "Topic Description": None,
    "Document Link": None,
    "Topic Keywords": None
}

def updateTopic(id, label, desc, link, key):
    Topics.update({"Topic ID": id})
    Topics.update({"Topic Label": label})
    Topics.update({"Topic Description": desc})
    Topics.update({"Document Link": link})
    Topics.update({"Topic Keywords": key})

    all_topics.append(Topics.copy())

In [ ]:
all_names = []

CVE_name = {
    "number": None,
    "name": None,
    "description": None,
}

def fillName(number, name, description):
    CVE_name.update({"number": number})
    CVE_name.update({"name": name})
    CVE_name.update({"description": description})
    all_names.append(CVE_name.copy())

In [ ]:
fillName("682", "Incorrect Calculation", "The software performs a calculation that generates incorrect or unintended results that are later used in security-critical decisions or resource management.")
fillName("688", "Exposure of Resource to Wrong Sphere", "The product exposes a resource to the wrong control sphere, providing unintended actors with inappropriate access to the resource.")
fillName("532", "Insertion of Sensitive Information into Log File", "Information written to log files can be of a sensitive nature and give valuable guidance to an attacker or expose sensitive user information.")
fillName("693", "Protection Mechanism Failure", "The product does not use or incorrectly uses a protection mechanism that provides sufficient defense against directed attacks against the product.")
fillName("369", "Divide By Zero", "The product divides a value by zero.")
fillName("131", "Incorrect Calculation of Buffer Size", "The software does not correctly calculate the size to be used when allocating a buffer, which could lead to a buffer overflow.")
fillName("788", "Access of Memory Location After End of Buffer", "The software reads or writes to a buffer using an index or pointer that references a memory location after the end of the buffer.")
fillName("121", "Stack-based Buffer Overflow", "A stack-based buffer overflow condition is a condition where the buffer being overwritten is allocated on the stack (i.e., is a local variable or, rarely, a parameter to a function).")
fillName("843", "Access of Resource Using Incompatible Type ('Type Confusion')", "The program allocates or initializes a resource such as a pointer, object, or variable using one type, but it later accesses that resource using a type that is incompatible with the original type.")
fillName("362", "Concurrent Execution using Shared Resource with Improper Synchronization ('Race Condition')", "The program contains a code sequence that can run concurrently with other code, and the code sequence requires temporary, exclusive access to a shared resource, but a timing window exists in which the shared resource can be modified by another code sequence that is operating concurrently.")
fillName("732", "Incorrect Permission Assignment for Critical Resource", "The product specifies permissions for a security-critical resource in a way that allows that resource to be read or modified by unintended actors.")
fillName("1021", "Improper Restriction of Rendered UI Layers or Frames", "The web application does not restrict or incorrectly restricts frame objects or UI layers that belong to another application or domain, which can lead to user confusion about which interface the user is interacting with.")
fillName("1188", "Insecure Default Initialization of Resource", "The software initializes or sets a resource with a default that is intended to be changed by the administrator, but the default is not secure.")
fillName("346", "Origin Validation Error", "The software does not properly verify that the source of data or communication is valid.")
fillName("59", "Improper Link Resolution Before File Access ('Link Following')", "The software attempts to access a file based on the filename, but it does not properly prevent that filename from identifying a link or shortcut that resolves to an unintended resource.")
fillName("611", "Improper Restriction of XML External Entity Reference", "The software processes an XML document that can contain XML entities with URIs that resolve to documents outside of the intended sphere of control, causing the product to embed incorrect documents into its output.")
fillName("295", "Improper Certificate Validation", "The software does not validate, or incorrectly validates, a certificate.")
fillName("770", "Allocation of Resources Without Limits or Throttling", "The software allocates a reusable resource or group of resources on behalf of an actor without imposing any restrictions on the size or number of resources that can be allocated, in violation of the intended security policy for that actor.")
fillName("119", "Improper Restriction of Operations within the Bounds of a Memory Buffer", "The software performs operations on a memory buffer, but it can read from or write to a memory location that is outside of the intended boundary of the buffer.")
fillName("190", "Integer Overflow or Wraparound", "The software performs a calculation that can produce an integer overflow or wraparound, when the logic assumes that the resulting value will always be larger than the original value. This can introduce other weaknesses when the calculation is used for resource management or execution control.")
fillName("129", "Improper Validation of Array Index", "The product uses untrusted input when calculating or using an array index, but the product does not validate or incorrectly validates the index to ensure the index references a valid position within the array.")
fillName("74", "Improper Neutralization of Special Elements in Output Used by a Downstream Component ('Injection')", "The software constructs all or part of a command, data structure, or record using externally-influenced input from an upstream component, but it does not neutralize or incorrectly neutralizes special elements that could modify how it is parsed or interpreted when it is sent to a downstream component.")
fillName("404", "Improper Resource Shutdown or Release", "The program does not release or incorrectly releases a resource before it is made available for re-use.")
fillName("502", "Deserialization of Untrusted Data", "The application deserializes untrusted data without sufficiently verifying that the resulting data will be valid.")
fillName("642", "External Control of Critical State Data", "The software stores security-critical state information about its users, or the software itself, in a location that is accessible to unauthorized actors.")
fillName("327", "Use of a Broken or Risky Cryptographic Algorithm", "The use of a broken or risky cryptographic algorithm is an unnecessary risk that may result in the exposure of sensitive information.")
fillName("294", "Authentication Bypass by Capture-replay", "A capture-replay flaw exists when the design of the software makes it possible for a malicious user to sniff network traffic and bypass authentication by replaying it to the server in question to the same effect as the original message (or with minor changes).")
fillName("200", "Exposure of Sensitive Information to an Unauthorized Actor", "The product exposes sensitive information to an actor that is not explicitly authorized to have access to that information.")
fillName("330", "Use of Insufficiently Random Values", "The software uses insufficiently random numbers or values in a security context that depends on unpredictable numbers.")
fillName("863", "Incorrect Authorization", "The software performs an authorization check when an actor attempts to access a resource or perform an action, but it does not correctly perform the check. This allows attackers to bypass intended access restrictions.")
fillName("347", "Improper Verification of Cryptographic Signature", "The software does not verify, or incorrectly verifies, the cryptographic signature for data.")
fillName("312", "Cleartext Storage of Sensitive Information", "The application stores sensitive information in cleartext within a resource that might be accessible to another control sphere.")
fillName("79", "Cross-site Scripting", "The software does not neutralize or incorrectly neutralizes user-controllable input before it is placed in output that is used as a web page that is served to other users.")
fillName("269", "Improper Priviledge Management", "The software does not properly assign, modify, track, or check privileges for an actor, creating an unintended sphere of control for that actor.")
fillName("20", "Improper Input Validation", "The product receives input or data, but it does not validate or incorrectly validates that the input has the properties that are required to process the data safely and correctly.")
fillName("310", "Cryptographic Issues", "Weaknesses in this category are related to the design and implementation of data confidentiality and integrity. Frequently these deal with the use of encoding techniques, encryption libraries, and hashing algorithms. The weaknesses in this category could lead to a degradation of the quality data if they are not addressed.")
fillName("89", "SQL Injection", "The software constructs all or part of an SQL command using externally-influenced input from an upstream component, but it does not neutralize or incorrectly neutralizes special elements that could modify the intended SQL command when it is sent to a downstream component.")
fillName("120", "Buffer Overflow", "The program copies an input buffer to an output buffer without verifying that the size of the input buffer is less than the size of the output buffer, leading to a buffer overflow.")
fillName("287", "Improper Authentication", "When an actor claims to have a given identity, the software does not prove or insufficiently proves that the claim is correct.")
fillName("416", "Use After Free", "Referencing memory after it has been freed can cause a program to crash, use unexpected values, or execute code.")
fillName("400", "Uncontrolled Resource Consumption", "The software does not properly control the allocation and maintenance of a limited resource, thereby enabling an actor to influence the amount of resources consumed, eventually leading to the exhaustion of available resources.")
fillName("434", "Unrestricted Upload of File with Dangerous Type", "The software allows the attacker to upload or transfer files of dangerous types that can be automatically processed within the product's environment.")
fillName("476", "Null Pointer Derefence", "A NULL pointer dereference occurs when the application dereferences a pointer that it expects to be valid, but is NULL, typically causing a crash or exit.")
fillName("22", "Path Traversal", "The software uses external input to construct a pathname that is intended to identify a file or directory that is located underneath a restricted parent directory, but the software does not properly neutralize special elements within the pathname that can cause the pathname to resolve to a location that is outside of the restricted directory.")
fillName("77", "Command Injection", "The software constructs all or part of a command using externally-influenced input from an upstream component, but it does not neutralize or incorrectly neutralizes special elements that could modify the intended command when it is sent to a downstream component.")
fillName("787", "Out-Of-Bounds Write", "The software writes data past the end, or before the beginning, of the intended buffer.")
fillName("444", "HTTP Request Smuggling", "When malformed or abnormal HTTP requests are interpreted by one or more entities in the data flow between the user and the web server, such as a proxy or firewall, they can be interpreted inconsistently, allowing the attacker to \"smuggle\" a request to one device without the other device being aware of it.")
fillName("78", "OS Command Injection", "The software constructs all or part of an OS command using externally-influenced input from an upstream component, but it does not neutralize or incorrectly neutralizes special elements that could modify the intended OS command when it is sent to a downstream component.")
fillName("601", "Open Redirect", "A web application accepts a user-controlled input that specifies a link to an external site, and uses that link in a Redirect. This simplifies phishing attacks.")
fillName("754", "Improper Check for Unusual or Exceptional Conditions", "The software does not check or incorrectly checks for unusual or exceptional conditions that are not expected to occur frequently during day to day operation of the software.")
fillName("284", "Improper Access Control", "The software does not restrict or incorrectly restricts access to a resource from an unauthorized actor.")
fillName("125", "Out-of-bounds Read", "The software reads data past the end, or before the beginning, of the intended buffer.")
fillName("306", "Missing Authentication for Critical Function", "The software does not perform any authentication for functionality that requires a provable user identity or consumes a significant amount of resources.")

In [ ]:
arr2 = []

diction2 = {
    "Document_ID": None,
    "Topic_ID": None,
    "Correlation_Value": None,
}

def update(id1, id2, value):
    diction2.update({"Document_ID": id1})
    diction2.update({"Topic_ID": id2})
    diction2.update({"Correlation_Value": value})

    arr2.append(diction2.copy())

# Group top 15 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]
#show
sent_topics_sorteddf_mallet

topics = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
all_numbers = []
counter = 0
index = 0
once = True
top_num = 8


for i in range(len(sent_topics_sorteddf_mallet)):
  if counter == 0:
    index = topics.pop(0)
  print("i", i)
  if sent_topics_sorteddf_mallet['Topic_Num'][i] == float(0): #float(0) ?
    print(sent_topics_sorteddf_mallet['Topic_Num'][i])
    print(sent_topics_sorteddf_mallet['Topic_Perc_Contrib'][i])
#     print(sent_topics_sorteddf_mallet['Keywords'][i])
#     print(sent_topics_sorteddf_mallet['Text'][i])
    for j in range(len(data)):
          if data[j] == sent_topics_sorteddf_mallet['Text'][i]:
            print(cwe[j])
            for n in all_names:
                if type(n) is not str:
                    if cwe[j] != "NVD-CWE-noinfo" and cwe[j] != "NVD-CWE-Other" and cwe[j] != "Unavailable":
                        if n['number'] == cwe[j].split("/")[5].split(".")[0]:
                            print(n["name"], n['number'])
                            all_numbers.append(n["number"])
    print()
  print("counter", counter)
  counter += 1
  if counter == 15:
    counter = 0
    all_numbers
    # Pass the split_it list to instance of Counter class.
    counter = Counter(all_numbers)
  
    # most_common() produces k frequently encountered
    # input values and their respective counts.
    most_occur = counter.most_common(4)
  
    print(most_occur)

    #updateTopic(index, n["name"], n["description"], cwe[j], sent_topics_sorteddf_mallet['Keywords'][i])
    once = False

# #fields = ["Topic ID", "Topic Label", "Topic Description", "Document Link", "Topic Keywords"]
# #filename = "/content/Sheet1.csv"

# #with open(filename2, 'w') as csvfile:
# #  writer = csv.DictWriter(csvfile, fieldnames = fields2)
# #  writer.writeheader()
# #  writer.writerows(arr2)

In [ ]:
updateTopic(8, "Cleartext Storage of Sensitive Information", "The application stores sensitive information in cleartext within a resource that might be accessible to another control sphere.", "http://cwe.mitre.org/data/definitions/312.html", "page, store, user, site, session, content, conduct, victim, reflect, field")

In [ ]:
fields2 = ["Topic ID", "Topic Label", "Topic Description", "Document Link", "Topic Keywords"]
filename2 = "/content/Sheet3.csv"

with open(filename2, 'w') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames = fields2)
  writer.writeheader()
  writer.writerows(all_topics)

In [ ]:
all_topics

In [ ]:
all_years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

interval = []

for year in all_years:
  num = 0
  num += interval[len(interval) - 1] if len(interval) is not 0 else 0
  num += years[str(year)]
  interval.append(num)

# interval
total_importance = []

row = sorted(ldamallet[corpus], key=lambda x: (x[0]))

# for i in range(len(row)):
#     temp_row.append(i)
#     for j in range(len[row[i]]):
#         temp_row.append(row[i][j])
# with open('Question_Information.csv', 'a',encoding='utf-8') as file:
#     writer = csv.writer(file)
#     temp_row = []

#     writer.writerow([])


# print(row[0][0][1]) #0.010582010582010583
# def importance(n, y):
#   return (n / y) * 100

# for k in range(len(all_years)):
#   topic_importance = [0]*20
#   for i in range(0 if all_years[k] == 2011 else interval[k - 1], interval[k]):
#     for j in range(20):
#       topic_importance[j] += row[i][j][1]
#   temp_year = [years[str(all_years[k])]]*20
#   year_importance = list(map(importance, topic_importance, temp_year))
#   total_importance.append(year_importance)

# total_importance

# yearsss = {
#     "2011": None,
#     "2012": None,
#     "2013": None,
#     "2014": None,
#     "2015": None,
#     "2016": None,
#     "2017": None,
#     "2018": None,
#     "2019": None,
#     "2020": None,
#     "2021": None,
# }

# #dict_arr_years = []

# for i in total_importance:
#   for j in range(11):
#     yearsss.update({str(all_years[j]): i[j]})
#   dict_arr_years.append(yearsss.copy())

# lastYear = ["2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021"]

# #dict_arr_years
# #lastYear

# # with open("/content/importance.csv", 'w') as csvfile:
# #   writer = csv.DictWriter(csvfile, lastYear)
# #   writer.writeheader()
# #   writer.writerows(dict_arr_years)
# with open("importance.csv", 'w') as csvfile:
#   writer = csv.DictWriter(csvfile, lastYear)
#   writer.writeheader()
#   writer.writerows(dict_arr_years)

In [ ]:
e = enumerate(ldamallet[corpus])

In [ ]:
def save_to_file(sorted_data):
    data = []
    for s in sorted_data:
        data.append(s[1])
    with open('importance.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([data[0], data[1], data[2],data[3],data[4],data[5],data[6],data[7],data[8],data[9],data[10],
                        data[11],data[12],data[13],data[14],data[15],data[16],data[17]])

count = 0
temp_row = []
for i, row in e:
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j in range(18):
            temp_row.append(row[j])
            
        sorted_r = sorted(temp_row)
        save_to_file(sorted_r)
        temp_row.clear()
        print(count)
        count+=1

Number of dominant descriptions per topic

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
print(df_dominant_topics)

------------------------------------------------------------------------------

Get best coherence

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=12, limit=45, step=3)

# Show graph
limit=45; start=12; step=3;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

------------------------------------------------------------